In [ ]:
import subprocess
import os
import zipfile
import arcgis
from arcgis.gis import GIS
from arcgis.features import Feature, FeatureLayer, FeatureCollection, FeatureSet
from arcgis.geometry.filters import intersects
from arcgis.geometry import Geometry, Point, Polyline, lengths
import csv
import geopandas as gpd
from shapely.geometry import shape, LineString, Polygon
from shapely.ops import transform
import pyproj
from functools import partial
from collections import defaultdict

In [ ]:
#Logging in
# There are two ways to log in. One would require a client ID, which you might not have access to 
# unless your GIS administrator gives you the right privileges. I had trouble logging in without a client ID, but you might not. 
# Try method 1 first, especially if you don't have a client ID. If that doesn't work... 
# Try getting the right privileges from your admin and proceed with method 2.

In [ ]:
# Method 2: Logging in with a client ID
gis_url= "https://your_org.maps.arcgis.com" # url to your org's arcGIS
your_client_id = "your_id"               # paste client ID here
gis = GIS(gis_url, client_id=your_client_id)
print("Successfully logged in as: " + gis.properties.user.username)

In [ ]:
# Makes circular buffers of a radius of your choice around at each CE end station

webmap_search = gis.content.search("title:ce", item_type="Web Map")
webmap_item = webmap_search[0]
webmap_data = webmap_item.get_data()

# Search for arms
arms = gis.content.search(
    "title: Idaho, zero AND tags: shapefile, layer AND owner:tansar27_amherstcollege",
    item_type="Feature Layer",
    max_items=10000
)

for arm in arms:
    input_arm_url = arm.layers[0].url
    input_arm_lyr = FeatureLayer(input_arm_url)
    input_arm = input_arm_lyr.query(
        where="1=1", out_fields="*", out_sr=4326, return_geometry=True
    ).features

    # Collect last vertices as point features, keeping attributes
    end_pts = []
    for feature in input_arm:
        last_pt_coords = feature.geometry["paths"][-1][-1]
        last_pt_geom = Point({
            "x": last_pt_coords[0],
            "y": last_pt_coords[1],
            "spatialReference": {"wkid": 4326}
        })
        end_st_attrs = feature.attributes.copy()
        end_st_attrs['is_corner'] = False
        new_feature = Feature(geometry=last_pt_geom, attributes=end_st_attrs)
        end_pts.append(new_feature)
        if feature.attributes['Name'] == 'y arm':
            corner_pt_coords = feature.geometry["paths"][0][0]
            corner_pt_geom = Point({
                "x": corner_pt_coords[0],
                "y": corner_pt_coords[1],
                "spatialReference": {"wkid": 4326}
            })
            corner_attrs = feature.attributes.copy()
            corner_attrs['is_corner'] = True
            corner_feature = Feature(geometry=corner_pt_geom, attributes=corner_attrs)
            end_pts.append(corner_feature)
    
    # Buffer parameters
    diameter = 10  
    units = "Kilometers"
    end_type = "Flat"
    circle_name = (
        arm.title.replace(" ", "_").replace(".", "_").replace("-", "_")
        + f"_{diameter}km_Station_Buffer")

    # Skip if buffer already exists
    existing_circles = gis.content.search(f'title:"{circle_name}"', max_items=1)
    if existing_circles:
        print(f"{circle_name} already exists. Skipping.")
        continue
       
    endpt_attrs = end_pts[0].attributes
   
    # Need the buffers to keep the attributes of each of their associated arms in the orignal KML, so build a field definition
    fields = [{"name": "ObjectID", "type": "esriFieldTypeOID", "alias": "ObjectID"}]
    for key, val in endpt_attrs.items():
        # Guess field type based on value type
        if isinstance(val, int):
            ftype = "esriFieldTypeInteger"
        elif isinstance(val, float):
            ftype = "esriFieldTypeDouble"
        else:
            ftype = "esriFieldTypeString"
   
        fields.append({"name": key, "type": ftype, "alias": key})
     
     # Build FeatureCollection because buffer creation function needs FeatureCollection as an input paramater
    fc = FeatureCollection({
        "layers": [
            {
                "geometryType": "esriGeometryPoint",
                "objectIdField": "ObjectID",
                "fields": fields,
                "features": [feat.as_dict for feat in end_pts]
            }
        ]
    })

    # create buffers around all points
    circle_result = arcgis.features.use_proximity.create_buffers(
        input_layer=fc,
        distances=[diameter],
        units=units,
        end_type=end_type,
        output_name=circle_name
    )
    circle_url = circle_result.layers[0].url

    # Add buffer to the web map
    circle_layer = {
        "id": circle_name,
        "title": circle_name,
        "url": circle_url,
        "visibility": True,
        "opacity": 1,
        "layerType": "ArcGISFeatureLayer"
    }
    webmap_data["operationalLayers"].append(circle_layer)
    webmap_item.update(data=webmap_data)
    print(f"Added {circle_name} to web map.")

print(f"Web map '{webmap_item.title}' successfully updated with buffer layers.")

In [ ]:
circles = gis.content.search(
    f"title: 10km, zero, Idaho, Buffer AND owner:{gis.properties.user.username}",
    item_type="Feature Layer",
    max_items=10000
)
grav_roads = []
local_roads_url = "https://services2.arcgis.com/FiaPA4ga0iQKduv3/arcgis/rest/services/Transportation_v1/FeatureServer/8"
local_roads = FeatureLayer(local_roads_url)

# Project the latitude and longitude to a coordinate system that allows for distance measurements.
# This is a function for azimuthal equidistant projection centered on the centroid of the buffer.  
def to_local_aeqd_transformer(lon, lat):
    aeqd = pyproj.CRS.from_proj4(
        f"+proj=aeqd +lat_0={lat} +lon_0={lon} +datum=WGS84 +units=m +no_defs"
    )
    wgs84 = pyproj.CRS.from_epsg(4326)
    fwd = pyproj.Transformer.from_crs(wgs84, aeqd, always_xy=True).transform
    return fwd

circle_title_groups = defaultdict(list)
gravel_roads = []
for circle in circles:
    circle_title = circle.title
    input_circle = circle.layers[0]
    circle_features = input_circle.query(
        where="1=1", out_fields="*", out_sr=4326, return_geometry=True
    ).features
    for feat in circle_features:
        feat.attributes["circle_title"] = circle_title
        circle_title_groups[circle_title].append(feat)

for circle_title, features in circle_title_groups.items():
   # Group arms by tot_score, vert_x, and vert_y because these attributes exactly match for pairs of arms
    buffer_groups = defaultdict(list)
    for feat in features:
        attrs = feat.attributes
        key = (attrs["tot_score"], attrs["vert_x"], attrs["vert_y"])
        buffer_groups[key].append(feat)

    for site_index, ((tot_score, vert_x, vert_y), group_feats) in enumerate(buffer_groups.items(), start=1):
        for feat in group_feats:
            geom = Geometry(feat.geometry)
            simplified_geoms = arcgis.geometry.generalize(spatial_ref=4326, geometries=[geom], max_deviation=0.001, future=False)
            simplified_geom = simplified_geoms[0]
            poly = shape(simplified_geom)

            # Project buffer into a coordinate system appropriate for distance measurements
            centroid = poly.centroid
            fwd = to_local_aeqd_transformer(centroid.x, centroid.y)
            poly_m = transform(fwd, poly)

            roads_in_buffer = local_roads.query(
                geometry_filter=intersects(simplified_geom),
                out_fields="*",
                return_geometry=True,
                out_sr=4326
            )

            total_length_km = 0.0
            for road in roads_in_buffer.features:
                r_shape = shape(Geometry(road.geometry))
                r_m = transform(fwd, r_shape)
                clipped = r_m.intersection(poly_m)
                total_length_km += clipped.length / 1000.0

            if feat.attributes["is_corner"] == 0:
                arm = feat.attributes["Name"]
                station_name = f"{site_index}_{int(tot_score)}_{circle_title[:-15]}_{arm}_end_station"
                print("==========")
                print(f"{station_name}: {total_length_km:.3f} km of gravel roads")
                print("----------")
            elif feat.attributes["is_corner"] == -1:
                station_name = f"{site_index}_{int(tot_score)}_{circle_title[:-15]}_corner_station"
                print("==========")
                print(f"{station_name}: {total_length_km:.3f} km of gravel roads")
                print("----------")
            grav_roads.append([station_name, total_length_km])




In [ ]:
# output a csv of road distances

output_csv_grav = r"C:\Users\user\Downloads\CE_sites_data\Idaho_gravel_roads_10km.csv"

with open(output_csv_grav, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Station Name", "Length of Gravel Roads (km)"])
    writer.writerows(grav_roads)

print(f"CSV saved to {os.path.abspath(output_csv_grav)}")